
# Azure OpenAI Latency Test — Manhattan Life (Two-Row Header Flattening)

This notebook measures **LLM response time** for generating a **field-mapping rule-spec JSON** using your uploaded Manhattan Life CSV.

**Key features:**
- Uses your two-row header flattener to normalize headers from `/mnt/data/manhattan_life_raw_data.csv`.
- Sends **flattened headers** + your **carrier prompt** to Azure OpenAI once per trial.
- Reports mean/median/min/max/p95 latency and prints the last JSON rule-spec (truncated).


## 0) Environment — Azure OpenAI settings

In [ ]:

import os

# Set these to your actual values (or set them in your environment before running)
os.environ.setdefault("AZURE_OPENAI_API_KEY",     "<YOUR_API_KEY>")
os.environ.setdefault("AZURE_OPENAI_ENDPOINT",    "https://<your-resource>.openai.azure.com/")
os.environ.setdefault("AZURE_OPENAI_DEPLOYMENT",  "gpt-4o-mini")  # your chat deployment name
os.environ.setdefault("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")


## 1) Utilities — Two-Row Header Flattener (Provided)

In [ ]:

import re
import pandas as pd

## Load and flatten multi-row header
def flatten_two_header_csv(path: str) -> pd.DataFrame:
    tmp = pd.read_csv(path, header=None, dtype=str)
    tmp = tmp.fillna("")

    # First two rows are headers
    top = tmp.iloc[0].tolist()
    bottom = tmp.iloc[1].tolist()

    # Forward-fill across top header row
    ff = []
    last = ""
    for x in top:
        x = str(x).strip()
        if x:
            last = x
        ff.append(last)

    # Build merged column names
    cols = []
    for a, b in zip(ff, bottom):
        a = str(a).strip()
        b = str(b).strip()
        if not a and not b:
            name = "unnamed"
        elif not a:
            name = b
        elif not b:
            name = a
        else:
            name = f"{a} {b}"

        # Normalize names (remove spaces, slashes, periods)
        name = re.sub(r"\s+", " ", name)
        name = name.replace("/", "_").replace(".", "_").strip()
        name = re.sub(r"\s+", "_", name)
        cols.append(name)

    # Drop the two header rows, assign new columns
    df = tmp.iloc[2:].reset_index(drop=True)
    df.columns = cols

    # Drop empty columns
    df = df[[c for c in df.columns if not df[c].astype(str).str.strip().eq("").all()]]
    return df.fillna("")


## 2) Load & Inspect Flattened Headers

In [ ]:

from pathlib import Path

RAW_PATH = Path("/mnt/data/manhattan_life_raw_data.csv")
if not RAW_PATH.exists():
    raise FileNotFoundError(f"CSV not found: {RAW_PATH}")

df_flat = flatten_two_header_csv(RAW_PATH.as_posix())
CSV_HEADERS = list(df_flat.columns)

print("Detected flattened headers (count =", len(CSV_HEADERS), "):\n", CSV_HEADERS[:25], "...")
print("Data preview:")
df_flat.head()


## 3) Required destination fields & (paste) Manhattan Life prompt

In [ ]:

FINAL_COLUMNS = [
    "PolicyNo","PHFirst","PHLast","Status","Issuer","State","ProductType","PlanName",
    "SubmittedDate","EffectiveDate","TermDate","PaySched","PayCode","WritingAgentID",
    "Premium","CommPrem","TranDate","CommReceived","PTD","NoPayMon","Membercount"
]

# ⬇️ Paste your real Manhattan Life prompt text here
RULES_TEXT = """
You are a data transformation agent...
(Replace this with the exact text of carrier_prompts/manhattan_life_prompt.txt)
If a rule is unclear or TBD, return blank for that field.
Use 'PID' if the rule refers to PID but output needs PTD (executor will map PID→PTD).
Return STRICT JSON only. No prose.
"""


## 4) AzureChatOpenAI setup and latency benchmark

In [ ]:

import json, statistics, time

try:
    from langchain_openai import AzureChatOpenAI
except Exception:
    from langchain.chat_models import AzureChatOpenAI  # fallback

SYSTEM_PROMPT = """You are a data transformation agent.
Output JSON ONLY, no prose.
Return a mapping where keys are required output fields and values are objects:
{ "op": <one of [copy,const,date_mmddyyyy,date_plus_1m_mmddyyyy,name_first_from_full,name_last_from_full,money,membercount_from_commission,blank]>, 
  "source": <column name when applicable>, 
  "value": <for const> }.
If a rule says 'TBD' or 'blank' or is unclear, use {"op":"blank"}.
Never invent columns. Use exact source header strings when copying.
Use 'PID' if rules refer to PID but output needs PTD.
"""

def build_llm(timeout: int = 20, temperature: float = 1.0) -> AzureChatOpenAI:
    # Note: some Azure deployments lock temperature to 1.0; pass 1.0 to avoid 400 errors.
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT") or os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
    if not (api_key and endpoint and deployment):
        raise RuntimeError("Missing AZURE_OPENAI_* env vars")
    return AzureChatOpenAI(
        azure_deployment=deployment,
        api_version=api_version,
        temperature=temperature,
        request_timeout=timeout,
        max_retries=0  # raw latency
    )

def time_rule_spec_build(headers, rules_text, trials: int = 5, timeout: int = 20, temperature: float = 1.0):
    llm = build_llm(timeout=timeout, temperature=temperature)
    messages = [
        {"role":"system","content": SYSTEM_PROMPT},
        {"role":"user","content": json.dumps({
            "RequiredFields": FINAL_COLUMNS,
            "RawHeaders": headers,
            "RulesNarrative": rules_text,
            "OutputFormat": "Return STRICT JSON object keyed by RequiredFields (use 'PID' if rules say PID but output needs PTD). No prose."
        }, ensure_ascii=False)}
    ]
    latencies, outputs = [], []

    # Warmup (not counted)
    try:
        _ = llm.invoke(messages)
    except Exception as e:
        print("Warmup failed:", e)

    for i in range(trials):
        t0 = time.perf_counter()
        resp = llm.invoke(messages)
        t1 = time.perf_counter()
        latencies.append(t1 - t0)
        outputs.append(resp.content if hasattr(resp, "content") else str(resp))

    lat_sorted = sorted(latencies)
    p95 = lat_sorted[int(0.95*(len(lat_sorted)-1))] if latencies else None
    return {
        "trials": trials,
        "latencies_s": latencies,
        "mean_s": statistics.fmean(latencies) if latencies else None,
        "median_s": statistics.median(latencies) if latencies else None,
        "min_s": min(latencies) if latencies else None,
        "max_s": max(latencies) if latencies else None,
        "p95_s": p95,
        "last_output": outputs[-1] if outputs else ""
    }


## 5) Run the latency benchmark (using flattened headers)

In [ ]:

results = time_rule_spec_build(headers=CSV_HEADERS, rules_text=RULES_TEXT, trials=5, timeout=20, temperature=1.0)
print("Latency (seconds):")
print({k: v for k, v in results.items() if k not in ("latencies_s","last_output")})
print("\nRaw latencies per trial (s):", results["latencies_s"])
print("\nLast JSON output (truncated to 1200 chars):\n")
print(results["last_output"][:1200])


In [ ]:
import re
from difflib import get_close_matches

def _needs_source(op: str) -> bool:
    return op in {
        "copy","date_mmddyyyy","date_plus_1m_mmddyyyy",
        "name_first_from_full","name_last_from_full",
        "money","membercount_from_commission"
    }

def audit_rule_spec(df: pd.DataFrame, rule_spec: dict, topk: int = 3):
    headers = list(df.columns)
    headers_norm = {re.sub(r'[^a-z0-9]', '', h.lower()): h for h in headers}
    missing = []
    for tgt, spec in rule_spec.items():
        if tgt == "PID":  # will map to PTD
            pass
        op = spec.get("op","")
        if not _needs_source(op):
            continue
        src = (spec.get("source") or "").strip()
        if not src:
            missing.append((tgt, op, src, [], []))
            continue
        # exact (case-insensitive) check
        if src in headers or src.lower() in [h.lower() for h in headers]:
            continue
        # normalized check
        src_norm = re.sub(r'[^a-z0-9]', '', src.lower())
        if src_norm in headers_norm:
            continue
        # fuzzy suggestions
        candidates = get_close_matches(src, headers, n=topk) or []
        candidates2 = get_close_matches(src_norm, [re.sub(r'[^a-z0-9]','',h.lower()) for h in headers], n=topk)
        cand2_orig = []
        for c2 in candidates2:
            for h in headers:
                if re.sub(r'[^a-z0-9]','',h.lower()) == c2:
                    cand2_orig.append(h)
        missing.append((tgt, op, src, candidates, cand2_orig))
    return missing

# After you have df_flat and rule_spec:
missing = audit_rule_spec(df_flat, rule_spec)
for tgt, op, src, c1, c2 in missing:
    print(f"[MISSING] {tgt} <- ({op}) source='{src}'")
    if c1: print("  close (raw):", c1)
    if c2: print("  close (norm):", c2)


In [ ]:
import re
from typing import Dict

def _build_header_index(df: pd.DataFrame) -> Dict[str, str]:
    """
    Build a robust index: normalized header -> original header.
    Normalization: lowercase, remove non-alphanumerics.
    """
    idx = {}
    for h in df.columns:
        key = re.sub(r'[^a-z0-9]', '', str(h).lower())
        idx[key] = h
    return idx

def _get_col(df: pd.DataFrame, header_index: Dict[str, str], name: str) -> pd.Series:
    """
    Case/punctuation/underscore-insensitive lookup.
    Tries raw exact, case-insensitive, normalized match.
    """
    if not name:
        return pd.Series([""] * len(df), index=df.index, dtype="string")
    # raw exact
    if name in df.columns:
        return df[name].astype(str)
    # case-insensitive exact
    for h in df.columns:
        if h.lower() == name.lower():
            return df[h].astype(str)
    # normalized
    key = re.sub(r'[^a-z0-9]', '', name.lower())
    if key in header_index:
        return df[header_index[key]].astype(str)
    # give up
    return pd.Series([""] * len(df), index=df.index, dtype="string")


In [ ]:
## 1) Load rules from JSON (not inline), then compile
from pathlib import Path
import json

RULES_FILE = Path("carrier_prompts/manhattan_life_rules.json")
RULES_TEXT = RULES_FILE.read_text(encoding="utf-8")  # keep the rules outside the code

rule_spec = derive_rule_spec(
    required_fields=FINAL_COLUMNS,   # actually use the arg
    headers=list(df_flat.columns),
    rules_text=RULES_TEXT
)


In [ ]:
def derive_rule_spec(required_fields, headers, rules_text) -> dict:
    llm = build_llm(temperature=1.0)
    payload = {
        "RequiredFields": required_fields,   # <-- not FINAL_COLUMNS
        "RawHeaders": headers,
        "RulesNarrative": rules_text,
        "OutputFormat": "Return STRICT JSON object keyed by RequiredFields (use 'PID' if rules say PID but output needs PTD). No prose."
    }
    resp = llm.invoke([
        {"role":"system","content": SYSTEM_PROMPT},
        {"role":"user","content": json.dumps(payload, ensure_ascii=False)}
    ])
    return json.loads(resp.content)


In [ ]:
## 2) Make header lookup robust (spaces vs underscores, case, punctuation)
import re

def _build_header_index(df):
    # normalized header -> original header
    idx = {}
    for h in df.columns:
        k = re.sub(r'[^a-z0-9]', '', str(h).lower())
        idx[k] = h
    return idx

def _get_col(df, header_index, name: str):
    if not name:
        return pd.Series([""]*len(df), index=df.index, dtype="string")
    # exact
    if name in df.columns:
        return df[name].astype(str)
    # case-insensitive exact
    for h in df.columns:
        if h.lower() == name.lower():
            return df[h].astype(str)
    # normalized
    key = re.sub(r'[^a-z0-9]', '', name.lower())
    if key in header_index:
        return df[header_index[key]].astype(str)
    # not found
    return pd.Series([""]*len(df), index=df.index, dtype="string")


In [ ]:
def apply_rules_vectorized(df, rule_spec):
    header_index = _build_header_index(df)
    ...
    # replace previous calls with:
    src = _get_col(df, header_index, spec.get("source",""))


In [ ]:
## 3) Audit mismatches before transforming

from difflib import get_close_matches

def _needs_src(op:str) -> bool:
    return op in {"copy","date_mmddyyyy","date_plus_1m_mmddyyyy",
                  "name_first_from_full","name_last_from_full",
                  "money","membercount_from_commission"}

def audit_rule_spec(df, rule_spec, topk=3):
    headers = list(df.columns)
    norm = {re.sub(r'[^a-z0-9]','',h.lower()):h for h in headers}
    for tgt, spec in rule_spec.items():
        op = spec.get("op","")
        if not _needs_src(op): 
            continue
        src = (spec.get("source") or "").strip()
        if not src:
            print(f"[MISSING] {tgt} ({op}) source=''")
            continue
        if src in headers or any(h.lower()==src.lower() for h in headers):
            continue
        key = re.sub(r'[^a-z0-9]','',src.lower())
        if key in norm:
            continue
        c1 = get_close_matches(src, headers, n=topk)
        c2 = get_close_matches(key, list(norm.keys()), n=topk)
        c2 = [norm[k] for k in c2]
        print(f"[MISSING] {tgt} <- ({op}) source='{src}'")
        if c1: print("  close (raw):", c1)
        if c2: print("  close (norm):", c2)

audit_rule_spec(df_flat, rule_spec)


In [ ]:
## 5) Quick end-to-end (after you fix the rules file)
# 1) Flatten
df_flat = flatten_two_header_csv("/mnt/data/manhattan_life_raw_data.csv")

# 2) Build spec once
RULES_TEXT = Path("carrier_prompts/manhattan_life_rules.json").read_text(encoding="utf-8")
rule_spec = derive_rule_spec(FINAL_COLUMNS, list(df_flat.columns), RULES_TEXT)

# 3) Audit (should print nothing if all aligned)
audit_rule_spec(df_flat, rule_spec)

# 4) Transform
out_df = apply_rules_vectorized(df_flat, rule_spec)

# 5) Save
out_df.to_csv("/mnt/data/outbound/manhattan_life_standard.csv", index=False)
Path("/mnt/data/outbound/manhattan_life_standard.json").write_text(
    out_df.to_json(orient="records", force_ascii=False), encoding="utf-8"
)


In [ ]:
import re
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

# ---- Standard output schema (order preserved) ----
FINAL_COLUMNS = [
    "PolicyNo","PHFirst","PHLast","Status","Issuer","State","ProductType","PlanName",
    "SubmittedDate","EffectiveDate","TermDate","PaySched","PayCode","WritingAgentID",
    "Premium","CommPrem","TranDate","CommReceived","PTD","NoPayMon","Membercount"
]

# ---------- Robust header lookup ----------
def _build_header_index(df: pd.DataFrame) -> dict[str, str]:
    """
    Build a normalized header -> original header index.
    Normalization: lowercase, strip all non [a-z0-9].
    """
    idx: dict[str, str] = {}
    for h in df.columns:
        k = re.sub(r'[^a-z0-9]', '', str(h).lower())
        idx[k] = h
    return idx

def _get_col(df: pd.DataFrame, header_index: dict[str, str], name: str) -> pd.Series:
    """
    Case/underscore/punctuation insensitive column fetch.
    Returns empty-string Series if not found.
    """
    if not name:
        return pd.Series([""] * len(df), index=df.index, dtype="string")

    # 1) exact
    if name in df.columns:
        return df[name].astype(str)

    # 2) case-insensitive exact
    for h in df.columns:
        if h.lower() == name.lower():
            return df[h].astype(str)

    # 3) normalized
    key = re.sub(r'[^a-z0-9]', '', name.lower())
    if key in header_index:
        return df[header_index[key]].astype(str)

    # not found
    return pd.Series([""] * len(df), index=df.index, dtype="string")

# ---------- Vectorized helpers ----------
def _to_mmddyyyy(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce")
    return dt.dt.strftime("%m/%d/%Y").fillna("").astype("string")

def _add_one_month_mmddyyyy(s: pd.Series) -> pd.Series:
    dt = pd.to_datetime(s, errors="coerce")
    dtp = dt.apply(lambda x: x + relativedelta(months=1) if pd.notnull(x) else pd.NaT)
    return pd.Series(dtp).dt.strftime("%m/%d/%Y").fillna("").astype("string")

def _parse_case_name_first_last(series: pd.Series) -> tuple[pd.Series, pd.Series]:
    s = series.fillna("").astype(str).str.strip()
    comma = s.str.contains(",", regex=False)
    swapped = s.where(~comma, s.str.replace(",", "", regex=False).str.strip())

    def _normalize(name: str) -> str:
        if not name:
            return ""
        parts = name.split()
        if len(parts) >= 2:
            return " ".join(parts[1:] + parts[:1])  # move leading token to end
        return name

    normalized = swapped.where(~comma, swapped.map(_normalize))
    tokens = normalized.str.split()
    last = tokens.str[-1].fillna("")
    first = tokens.apply(lambda xs: " ".join(xs[:-1]) if isinstance(xs, list) and len(xs) > 1 else "").fillna("")
    return first.str.title().astype("string"), last.str.title().astype("string")

def _money_to_float_str(s: pd.Series) -> pd.Series:
    x = s.fillna("").astype(str).str.strip()
    neg_paren = x.str.match(r"^\(.*\)$")
    x = x.str.replace(r"[,$()]", "", regex=True).str.strip()
    x = np.where(neg_paren, "-" + x, x)
    num = pd.to_numeric(x, errors="coerce")
    return pd.Series(num.map(lambda v: f"{v:.2f}" if pd.notnull(v) else ""), index=s.index, dtype="string")

def _sign_flag_from_money(s: pd.Series) -> pd.Series:
    x = s.fillna("").astype(str).str.strip()
    neg_paren = x.str.match(r"^\(.*\)$")
    x = x.str.replace(r"[,$()]", "", regex=True).str.strip()
    x = np.where(neg_paren, "-" + x, x)
    num = pd.to_numeric(x, errors="coerce")
    out = np.where(pd.isna(num), "", np.where(num < 0, "-1", "1"))
    return pd.Series(out, index=s.index, dtype="string")

# ---------- Main compiler/executor ----------
ALLOWED_OPS = {
    "copy","const","date_mmddyyyy","date_plus_1m_mmddyyyy",
    "name_first_from_full","name_last_from_full",
    "money","membercount_from_commission","blank"
}

def apply_rules_vectorized(df: pd.DataFrame, rule_spec: dict) -> pd.DataFrame:
    """
    Apply LLM-compiled rule_spec to df and return a DataFrame with FINAL_COLUMNS.
    Supported spec entry for each target:
      {"op":"copy","source":"Col"}
      {"op":"const","value":"Active"}
      {"op":"date_mmddyyyy","source":"Col"}
      {"op":"date_plus_1m_mmddyyyy","source":"Col"}
      {"op":"name_first_from_full","source":"Col"}
      {"op":"name_last_from_full","source":"Col"}
      {"op":"money","source":"Col"}
      {"op":"membercount_from_commission","source":"Col"}  # "1" unless <0 → "-1"; blank→"1"
      {"op":"blank"}
    Special: if spec contains "PID", it is written to output "PTD".
    """
    header_index = _build_header_index(df)

    def empty() -> pd.Series:
        return pd.Series([""] * len(df), index=df.index, dtype="string")

    out: dict[str, pd.Series] = {}

    for tgt in FINAL_COLUMNS:
        # Allow alias: PID (rule) -> PTD (output)
        spec = rule_spec.get(tgt) or (rule_spec.get("PID") if tgt == "PTD" else None)
        if not spec or not isinstance(spec, dict):
            out[tgt] = empty()
            continue

        op = str(spec.get("op", "")).strip()
        if op not in ALLOWED_OPS:
            out[tgt] = empty()
            continue

        if op == "copy":
            out[tgt] = _get_col(df, header_index, spec.get("source", ""))
        elif op == "const":
            out[tgt] = pd.Series([str(spec.get("value", ""))] * len(df), index=df.index, dtype="string")
        elif op == "date_mmddyyyy":
            out[tgt] = _to_mmddyyyy(_get_col(df, header_index, spec.get("source", "")))
        elif op == "date_plus_1m_mmddyyyy":
            out[tgt] = _add_one_month_mmddyyyy(_get_col(df, header_index, spec.get("source", "")))
        elif op == "name_first_from_full":
            first, _ = _parse_case_name_first_last(_get_col(df, header_index, spec.get("source", "")))
            out[tgt] = first
        elif op == "name_last_from_full":
            _, last = _parse_case_name_first_last(_get_col(df, header_index, spec.get("source", "")))
            out[tgt] = last
        elif op == "money":
            out[tgt] = _money_to_float_str(_get_col(df, header_index, spec.get("source", "")))
        elif op == "membercount_from_commission":
            flags = _sign_flag_from_money(_get_col(df, header_index, spec.get("source", "")))
            out[tgt] = pd.Series(np.where(flags.eq(""), "1", flags), index=df.index, dtype="string")
        elif op == "blank":
            out[tgt] = empty()
        else:
            out[tgt] = empty()

    # Ensure order + types
    out_df = pd.DataFrame(out, columns=FINAL_COLUMNS).fillna("").astype("string")
    return out_df
